In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/processed/real_study_merged.csv")

In [3]:
df

,Unnamed: 0,Submission id,Participant id,Status,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,debrief.1.player.payoff,debrief.1.player.debrief,debrief.1.player.debrief2,debrief.1.group.id_in_subsession,debrief.1.subsession.round_number,thanks.1.player.id_in_group,thanks.1.player.role,thanks.1.player.payoff,thanks.1.group.id_in_subsession,thanks.1.subsession.round_number
0,0,630f36833742de51ded6cdc9,5ced7e9f3cad8c00179175af,AWAITING REVIEW,2022-08-31T10:27:37.977000Z,2022-08-31T10:36:30.058000Z,NaN,2022-08-31T10:36:30.395391Z,533.0,4BC08866,...,0.0,majority,sophisticated,1,1,22,NaN,0.0,1,1
1,1,630f369f890ea1ad65cff247,5bc6618e54b3bc0001a30ffa,AWAITING REVIEW,2022-08-31T10:23:27.540000Z,2022-08-31T10:35:43.742000Z,NaN,2022-08-31T10:35:44.133305Z,737.0,4BC08866,...,0.0,only_c,majority,1,1,3,NaN,0.0,1,1
2,2,630f36bb1dfca535f7b36228,5e6e6497d33339419afd90f9,AWAITING REVIEW,2022-08-31T10:23:55.318000Z,2022-08-31T10:30:48.805000Z,NaN,2022-08-31T10:30:49.187676Z,414.0,4BC08866,...,0.0,only_c,majority,1,1,4,NaN,0.0,1,1
3,3,630f36c40f784b284910283e,5ca606a5614b7b0015be495e,AWAITING REVIEW,2022-08-31T10:24:07.274000Z,2022-08-31T10:30:37.042000Z,NaN,2022-08-31T10:30:37.436077Z,390.0,4BC08866,...,0.0,random,only_c,1,1,5,NaN,0.0,1,1
4,4,630f36d6b12c0c04fd35ce37,5f24956d52c72509677c5d85,AWAITING REVIEW,2022-08-31T10:24:23.958000Z,2022-08-31T10:29:55.233000Z,NaN,2022-08-31T10:29:55.622101Z,332.0,4BC08866,...,0.0,minority,random,1,1,6,NaN,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,1992,630f5530c3ef4a7957a545b7,5f0766baa1aa2f12767758cd,AWAITING REVIEW,2022-08-31T12:33:52.599000Z,2022-08-31T12:42:06.312000Z,NaN,2022-08-31T12:42:06.769202Z,494.0,4BC08866,...,0.0,only_c,sophisticated,1,1,2247,NaN,0.0,1,1
1993,1993,630f5536ab62618e97a30fd9,5c61598fb40f380001109ca5,AWAITING REVIEW,2022-08-31T12:34:02.002000Z,2022-08-31T12:40:03.288000Z,NaN,2022-08-31T12:40:04.862300Z,362.0,4BC08866,...,0.0,random,sophisticated,1,1,2248,NaN,0.0,1,1
1994,1994,630f555a6deeab9d3377f2ab,562cee16733ea00011163376,AWAITING REVIEW,2022-08-31T12:34:39.096000Z,2022-08-31T12:43:27.460000Z,NaN,2022-08-31T12:43:27.892638Z,529.0,4BC08866,...,0.0,sophisticated,majority,1,1,2251,NaN,0.0,1,1
1995,1995,630f56048d9db4c676f5aa6d,627125a4c8bbe00988850330,AWAITING REVIEW,2022-08-31T12:38:40.824000Z,2022-08-31T12:46:00.635000Z,NaN,2022-08-31T12:46:01.042585Z,440.0,4BC08866,...,0.0,minority,majority,1,1,2253,NaN,0.0,1,1


In [4]:
df.shape

(1997, 185)

In [5]:
# transform a dataframe with data in multiple columns into a dataframe with data in a single column
decision_cols = [col for col in df.columns if 'player.decision' in col]
decision_cols.insert(0, 'Participant id')

time_columns = [c for c in df.columns if 'time_spent' in c]
time_columns.insert(0, 'Participant id')

In [6]:
decisions_panel = df[decision_cols].set_index('Participant id').stack().reset_index().rename(columns={'level_1': 'Round', 0: 'Decision'})
time_panel = df[time_columns].set_index('Participant id').stack().reset_index().rename(columns={'level_1': 'Round', 0: 'TimeSpent'})

In [7]:
decisions_panel.shape

(21967, 3)

In [8]:
decisions_panel["Round"] = decisions_panel["Round"].str.replace(".*\.(\d+)\..*", lambda m: m.group(1), regex=True)
time_panel["Round"] = time_panel["Round"].str.replace(".*\.(\d+)\..*", lambda m: m.group(1), regex=True)

In [9]:
panel_df = pd.merge(left=time_panel, left_on=["Participant id", "Round"], right=decisions_panel, right_on=["Participant id", "Round"])

In [10]:
covariate_cols = ['Participant id', "bret.1.player.boxes_collected", "debrief.1.player.debrief", "debrief.1.player.debrief2", 'Total approvals', 'Total rejections', 'Approval rate', 'Negotiation experience', 'Highest education level completed', 'Deception', 'Charitable giving', 'Age', 'Sex', 'Ethnicity simplified', 'Country of birth',
       'Country of residence', 'Nationality', 'Language', 'Student status',
       'Employment status', 'participant.in_deception']

In [11]:
df_covars = df[covariate_cols].copy()

In [12]:
panel_df = pd.merge(panel_df, df_covars, on='Participant id')

In [13]:
panel_df['Round'] = panel_df['Round'].astype(int)

In [14]:
strategy_schedule = [18, 18, 16, 14, 12, 10, 8, 6, 4, 2, 2, 2]

In [47]:
strategy_df = pd.Series({(i+1): 20-v for i, v in enumerate(strategy_schedule)}, name='n_cooperators')

In [49]:
panel_df = panel_df.merge(strategy_df, left_on='Round', right_index=True, how="left")

In [50]:
panel_df['prop_coop'] = panel_df['n_cooperators'] / 20

In [51]:
panel_df.to_csv("../data/processed/panel_data.csv", index=False, )

In [52]:
panel_df.shape

(21967, 28)

In [53]:
panel_df.to_stata("../data/processed/panel_data.dta")

/var/folders/6k/5hmkfvg15c96n_sg9wvgtgzm0000gn/T/ipykernel_80748/106464356.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Participant id   ->   Participant_id
    bret.1.player.boxes_collected   ->   bret_1_player_boxes_collected
    debrief.1.player.debrief   ->   debrief_1_player_debrief
    debrief.1.player.debrief2   ->   debrief_1_player_debrief2
    Total approvals   ->   Total_approvals
    Total rejections   ->   Total_rejections
    Approval rate   ->   Approval_rate
    Negotiation experience   ->   Negotiation_experience
    Highest education level completed   ->   Highest_education_level_complete
    Charitable giving   ->   Charitable_giving
    Ethnicity simplified   ->   Ethnicity_simplified
    Country of birth   ->   Country_of_birth
    Country of residence   ->   Country_of_residence
    Student status   ->   Student_status
    Employment status   ->   Employment_status
    partic